In [3]:
import pyodbc
import pandas as pd
import sqlite3 as sqlite
import sqlite3
from sqlite3 import Error


In [14]:
#1. Código de persistência de dados

try:

#1.1. Consulta SQL server - Benner    
    conn_pyodbc = pyodbc.connect(r'Driver={SQL Server};Server=CSD0201WNT62V;Database=BENNERAG;Trusted_Connection=yes;')
    sql_guia_eventos = pd.read_sql_query('''--*** ESTA CONSULTA RETORNA OS PAGAMENTOS DE EVENTOS ESPECIFICOS. INCLUSIVE MATERIAL SUBITEM

        Select 
        pr.nome as PRESTADOR,
        p.peg as PEG, 
        g.GUIAOPERADORA as GUIA,
        ge.TABELAPRECO,
        ge.qtdpagto,
        ge.VALORAPRESENTADO,
        ge.VALORPAGTO,
        ge.DATAATENDIMENTO,
        ge.HORAATENDIMENTO,
        ge.VALORCALCPAGTO,
        ge.grau,
        ge.SITUACAO,
        ge.ORIGEMEVENTO,
        ge.codigodesprealizadas,
        --m.codigoitem,
        t.DESCRICAO as tdescricao,
        t.ESTRUTURA,
        p.DATAPAGAMENTO,
        est.nome,
        gr.descricao
        --samgl.MOTIVOGLOSA
        --m.dataatendimento

        from sam_peg p 
        left join sam_guia g on g.peg = p.handle 
        left join sam_guia_eventos ge on ge.guia = g.handle
        --left join SAM_GUIA_EVENTOS_MATMED m on m.GUIAEVENTO = ge.handle
        --left join empresas e on e.handle = p.empresa
        left join sam_prestador pr on pr.handle = p.recebedor
        join sam_tge t on t.handle = ge.evento
        join sam_beneficiario b on b.handle = g.beneficiario
        join sam_grau gr on gr.handle = ge.grau
        left join estados est on pr.ESTADOPAGAMENTO = est.HANDLE
        --full outer join SAM_GUIA_EVENTOS_GLOSA samgl on samgl.GUIAEVENTO = ge.GUIA
        --left join SAM_MOTIVOGLOSA mgl on samgl.MOTIVOGLOSA = mgl.HANDLE
        --where p.datarecebimento >= '2019-01-01' and (t.ESTRUTURA like '%6.00.00.953%' OR t.ESTRUTURA like '%6.00.01.062%')

        --where p.datarecebimento >= '2019-01-01'

        --where p.datarecebimento >= '2020-01-01' and (t.DESCRICAO like '%MISTURA DE GASES NO RESPIRADOR/VENTILADOR, POR HORA%' or t.DESCRICAO like '%DIÁRIA DE ISOLAMENTO DE UTI ADULTO GERAL%')
        where p.datarecebimento >= '2020-01-01' and (ge.codigodesprealizadas = 1 or ge.codigodesprealizadas = 5) order by t.estrutura
        ''', conn_pyodbc)
    
except pyodbc.OperationalError as error:
    
     print("Erro ao tentar conectar o SQL Server", error)
        
finally:
    if (conn_pyodbc):
        conn_pyodbc.close()
        print("A conexão do SQL server foi fechada")

#1.2. Conexão SQLite

try:

    sqliteConnection = sqlite3.connect('Indicador_GasesDB.db')
    cursor = sqliteConnection.cursor()
    print("Database criada e conexão bem sucedida ao SQLite")
    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    print("SQLite version is: ", record)

    sql_guia_eventos.to_sql('Gases_diarias', sqliteConnection, if_exists="replace")
    
    cursor.close()

except sqlite3.Error as error:
    print("Erro ao tentar conectar SQLite", error)

finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("Conexão SQLite fechada")

A conexão do SQL server foi fechada
Database criada e conexão bem sucedida ao SQLite
SQLite version is:  [('3.32.3',)]
Conexão SQLite fechada


In [15]:
try:

    sqliteConnection = sqlite3.connect('Indicador_GasesDB.db')
    cursor = sqliteConnection.cursor()
    print("Database criada e conexão bem sucedida ao SQLite")
    sqlite_select_Query = "select sqlite_version();"
    cursor.execute(sqlite_select_Query)
    record = cursor.fetchall()
    print("SQLite version is: ", record)

    sql_guia_eventosQuery = pd.read_sql("select * from Gases_diarias", con=sqliteConnection)
    
    cursor.close()

except sqlite3.Error as error:
    print("Erro ao tentar conectar SQLite", error)

finally:
    if (sqliteConnection):
        sqliteConnection.close()
        print("Conexão SQLite fechada")
        
sql_guia_eventosQuery

Database criada e conexão bem sucedida ao SQLite
SQLite version is:  [('3.32.3',)]
Conexão SQLite fechada


,index,PRESTADOR,PEG,GUIA,TABELAPRECO,qtdpagto,VALORAPRESENTADO,VALORPAGTO,DATAATENDIMENTO,HORAATENDIMENTO,VALORCALCPAGTO,grau,SITUACAO,ORIGEMEVENTO,codigodesprealizadas,tdescricao,ESTRUTURA,DATAPAGAMENTO,nome,descricao
0,0,SANTA CASA DE MISERICORDIA DE SOBRAL,2.020189e+09,12144836,157.0,0.0,275.65,0.0,2020-01-18 00:00:00,1899-12-30 14:40:00,0.00,20,4,N,5,DIARIA APTO C/ISOLAMENTO,0.80.01.2027,2020-04-22 00:00:00,CEARÁ,Diaria de Apartamento
1,1,HOSPITAL DE AVILA LTDA,2.020238e+09,32238460,156.0,0.0,343.79,0.0,2020-03-13 00:00:00,1899-12-30 17:01:00,0.00,20,4,N,5,DIARIA APTO PADRAO,0.80.01.2108,2020-05-21 00:00:00,PERNAMBUCO,Diaria de Apartamento
2,2,QLVX SAUDE EXTRA HOSPITALAR LTDA EP,1.901401e+07,32327661,NaN,10.0,1488.00,1488.0,2020-05-01 00:00:00,1899-12-30 08:00:00,1543.37,1486,4,N,5,DIARIA DE INTERNACAO EM CASA COM ASSIST DE 12HS,0.80.01.9471,2020-07-21 00:00:00,BAHIA,Internação em Casa (Dotações)
3,3,QLVX SAUDE EXTRA HOSPITALAR LTDA EP,2.020127e+09,12096799,157.0,1.0,148.80,148.8,2020-01-21 00:00:00,1899-12-30 08:00:00,154.34,1486,4,N,5,DIARIA DE INTERNACAO EM CASA COM ASSIST DE 12HS,0.80.01.9471,2020-03-23 00:00:00,BAHIA,Internação em Casa (Dotações)
4,4,QLVX SAUDE EXTRA HOSPITALAR LTDA EP,1.903479e+07,32496901,NaN,0.0,3720.00,0.0,2020-08-07 00:00:00,1899-12-30 08:00:00,0.00,28,2,N,5,DIARIA DE INTERNACAO EM CASA COM ASSIST DE 12HS,0.80.01.9471,2020-10-21 00:00:00,BAHIA,Internação em Casa (Valor Informado)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27340,27340,LISIENE SIEBRA & CIA LTDA,1.902500e+07,32424835,NaN,0.0,0.03,0.0,2020-07-29 00:00:00,1899-12-30 00:00:00,0.00,1308,4,N,1,Cateter Balão para PTA (Angioplastia Translumi...,8.00.91.059,2020-09-21 00:00:00,CEARÁ,Materiais (Dotacoes)
27341,27341,PRONTOCARDIO-PRONTO ATEND CARD S/C LTDA,2.020243e+08,12019012,159.0,20.0,7.00,7.0,2019-11-19 00:00:00,1899-12-30 17:29:00,0.00,39,4,N,1,LACTULONA - 667 MG/ML XPE CT FR PLAST (PET) AM...,9.00.49.560,2020-02-21 00:00:00,CEARÁ,Medicamentos (Valor Informado)
27342,27342,PRONTOCARDIO-PRONTO ATEND CARD S/C LTDA,2.020243e+08,12019012,159.0,20.0,7.00,7.0,2019-11-18 00:00:00,1899-12-30 17:14:00,0.00,39,4,N,1,LACTULONA - 667 MG/ML XPE CT FR PLAST (PET) AM...,9.00.49.560,2020-02-21 00:00:00,CEARÁ,Medicamentos (Valor Informado)
27343,27343,PRONTOCARDIO-PRONTO ATEND CARD S/C LTDA,2.020243e+08,12019012,159.0,10.0,3.50,3.5,2019-11-17 00:00:00,1899-12-30 17:29:00,0.00,39,4,N,1,LACTULONA - 667 MG/ML XPE CT FR PLAST (PET) AM...,9.00.49.560,2020-02-21 00:00:00,CEARÁ,Medicamentos (Valor Informado)


In [16]:
#2. classifica os eventos em oxigênio e diárias
#sql_guia_eventos.info()

#2.1. Cria série única de códigos dos eventos (Não funciona ao fazer merge com a base completa):
df_classificaEventos = pd.DataFrame(data=None)
df_classificaEventos['ESTRUTURA'] = pd.Series(sql_guia_eventosQuery['ESTRUTURA'].unique())
df2 = pd.merge(df_classificaEventos, sql_guia_eventos, left_on='ESTRUTURA', right_on='ESTRUTURA')

#Classifica os eventos em diárias e unidade de contagem (dia, hora e minuto)
searchStringTrue = 'True'
searchStringFlase = 'False'
searchStringDiariainternacao = ['diaria', 'diária']
searchStringDia = ['dia', 'diá']
searchStringHora = 'hora'
searchStringMinuto = 'minuto'
searchStringEventosgases = ['6.00.34.335',
'6.00.28.378',
'6.00.34.122',
'6.00.28.475',
'6.00.28.467',
'6.00.28.599',
'6.00.28.335',
'6.00.28.394',
'6.00.11.017',
'6.00.04.762',
'6.00.04.614',
'6.00.28.432',
'6.00.28.440',
'6.00.04.746',
'6.00.10.770',
'6.00.34.343',
'6.00.04.592',
'6.00.28.343',
'6.00.28.491',
'6.00.11.009',
'6.00.10.851',
'6.00.28.459',
'6.00.02.824',
'6.00.04.754',
'6.00.04.606',
'6.00.04.460',
'6.00.34.157',
'6.00.04.894',
'6.00.08.016',
'6.00.11.246',
'6.00.11.254',
'6.00.11.092',
'6.00.34.130',
'6.00.11.106',
'6.00.05.866',
'6.00.28.351',
'6.00.06.013',
'6.00.10.045',
'6.00.27.738',
'6.00.28.564',
'0.82.04.1067',
'6.00.11.084',
'0.80.08.2033',
'6.00.28.521',
'6.00.04.444',
'6.00.08.318',
'6.00.02.662',
'6.00.10.053',
'6.00.34.327',
'6.00.04.142',
'6.00.04.312',
'0.80.08.1240',
'6.00.34.238',
'6.00.04.452',
'6.00.04.010',
'6.00.04.169',
'6.00.27.762',
'6.00.25.484',
'6.00.06.021',
'6.00.28.513',
'6.00.10.452',
'6.00.34.106',
'6.00.11.327',
'6.00.34.483',
'6.00.26.391',
'6.00.04.371',
'0.82.02.1031',
'6.00.02.816',
'6.00.05.874',
'6.00.34.300',
'6.00.11.335',
'6.00.34.203',
'6.00.32.120',
'6.00.03.847',
'0.80.08.1940',
'6.00.05.882',
'6.00.10.207',
'6.00.10.444',
'6.00.34.211',
'6.00.31.808',
'6.00.32.081',
'0.80.08.1304',
'0.80.08.1630',
'6.00.06.048',
'6.00.28.483',
'6.00.34.165',
'6.00.26.332',
'6.00.01.860',
'6.00.02.263',
'6.00.03.065',
'6.00.04.002',
'6.00.04.150',
'6.00.04.290',
'6.00.04.304',
'0.80.08.1290',
'0.80.08.1690',
'0.80.08.1711',
'0.80.08.1908',
'0.80.08.2165',
'6.00.05.890',
'6.00.06.790',
'6.00.09.977',
'6.00.08.326',
'6.00.08.334',
'6.00.09.063',
'6.00.34.297',
'6.00.10.843',
'6.00.14.911',
'6.00.27.681']

#Padroniza os arrays de palavras chave para a função .contains
searchStringDiariainternacao_pattern = '|'.join(searchStringDiariainternacao)
searchStringDia_pattern = '|'.join(searchStringDia)
searchStringEventosgases_pattern = '|'.join(searchStringEventosgases)

#Padroniza a descrião para minusculo
df2['Lower tdescricao'] = df2['tdescricao'].str.lower()

#Pesquisa as palvras chave e classifica entre evento de dia ou gases
df2['Flag diarias'] = df2['Lower tdescricao'].str.contains(searchStringDiariainternacao_pattern)
df2['Flag gases'] = df2['ESTRUTURA'].str.contains(searchStringEventosgases_pattern)

#Atribui o valor de consumo por hora
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringDia_pattern) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 24
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringHora) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 1
df2.loc[(df2['Lower tdescricao'].str.contains(searchStringMinuto) == True) & (df2['Flag gases'] == True), 'Unidade gases'] = 1/60

#Classifica em evento válido (gases ou diárias)
df2.loc[(df2['Flag gases']  == True) | (df2['Flag diarias'] == True), 'Evento valido'] = 1

#Consumo em horas de gases e diárias, por evento
df2.loc[df2['Flag diarias']  == True, 'Diárias (horas)'] = 24*df2['qtdpagto']
df2.loc[(df2['Flag gases']  == True), 'Consumo de gases (horas)'] = df2['Unidade gases']*df2['qtdpagto']

#Groupby
#df2['Consumo total de dias (em horas)'] = df2.groupby('GUIA')['Diárias (horas)'].cumsum()
#df2['Consumo total de gases (em horas)'] = df2.groupby('GUIA')['Consumo de gases (horas)'].cumsum()

#df2.loc[(df2['Consumo total de dias (em horas)', total].sum()

#3. Cria pivottable para comparação do total de horas entre gases e diárias
df2_pivot = df2.pivot_table(values = ['Diárias (horas)', 'Consumo de gases (horas)'], index = ['GUIA', 'PRESTADOR'], aggfunc = 'sum')

df2_pivot.loc[(df2_pivot['Consumo de gases (horas)'] > df2_pivot['Diárias (horas)']), 'Consumo maior'] = "Sim"
df2_pivot.loc[(df2_pivot['Consumo de gases (horas)'] < df2_pivot['Diárias (horas)']), 'Consumo maior'] = "Não"
df2_pivot.loc[(df2_pivot['Consumo de gases (horas)'] == df2_pivot['Diárias (horas)']), 'Consumo maior'] = "Igual"

df2_pivot


,,Consumo de gases (horas),Diárias (horas),Consumo maior
GUIA,PRESTADOR,,,
11967114,SAUDE BRB - CAIXA DE ASSISTENCIA,0.0,96.0,Não
11967117,SAUDE BRB - CAIXA DE ASSISTENCIA,0.0,120.0,Não
11967118,SAUDE BRB - CAIXA DE ASSISTENCIA,0.0,120.0,Não
11967122,SAUDE BRB - CAIXA DE ASSISTENCIA,0.0,120.0,Não
11967124,SAUDE BRB - CAIXA DE ASSISTENCIA,0.0,96.0,Não
...,...,...,...,...
32509889,HOSPITAL ESPERANCA S/A,3.0,72.0,Não
32510084,ESHO EMPRESA DE SERVICOS HOSPITALARES S.A,1.2,0.0,Sim
32510130,SANTA CASA DE MISERICORDIA DA BAHIA,2.0,24.0,Não
